In [2]:
from Bio import SeqIO
import os,sys
import pandas as pd
import phylopandas as ph

In [3]:

def getgene(fasta_file, anno_list):
    f = open(fasta_file, "w")
    for rec in SeqIO.parse("BetaCoV_0529.gb", "gb"):
        for feature in rec.features:
            for key, val in feature.qualifiers.items():
                if feature.type == "CDS":
                    if any (s in val for s in anno_list):
                        print (">" + rec.id, file=f )
                        print (feature.location.extract(rec).seq,file=f)
    f.close()

In [3]:
def duplicate_remover(fasta_file):
    df = ph.read_fasta(fasta_file)
    df = df.filter(['id','sequence'], axis=1)
    df = df.drop_duplicates()
    df.to_csv("temp.tab", sep="\t",index = False,header=False)
    df2= SeqIO.parse("temp.tab", "tab")
    SeqIO.write(df2, "rm"+fasta_file, "fasta")
    os.remove("temp.tab")
    print("duplicate records removed!\n"+ str(len(df.index))+" unique records saved from " + fasta_file)

In [5]:
## process annotation table 

ano = pd.read_csv('BetaCoV_0529 Annotations.csv')
CDS = ano.loc[ano['Type'] == 'CDS']
CDS_name = CDS.groupby(['Name']).count()
CDS_name

/Users/jianichen1/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Sequence Name,Type,Minimum,Maximum,Length,# Intervals,Direction,Sequence (with extension),Length (with extension)
Name,,,,,,,,,
1 CDS,13,13,13,13,13,13,13,0,0
10.1 kDa nonstructural protein CDS,1,1,1,1,1,1,1,0,0
12.6 kD nonstructural protein CDS,2,2,2,2,2,2,2,0,0
12.7 kD non-structural protein CDS,5,5,5,5,5,5,5,0,0
12.7 kDa accessory protein CDS,1,1,1,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...
unnamed protein product; orf3a CDS,1,1,1,1,1,1,1,0,0
unnamed protein product; orf3b CDS,1,1,1,1,1,1,1,0,0
unnamed protein product; orf6 CDS,1,1,1,1,1,1,1,0,0


In [6]:

CDS_name.to_csv("CDSname_0529.csv")

In [17]:
S={'S',
'spike protein',
'S protein',
'spike glycoprotein',
'spike glycoprotein precursor',
'spike',
'membrane glycoprotein',
'spike glycoprotein S',
'putative spike glycoprotein',
'Spike protein',
'putative E2 glycoprotein precursor',
'spike surface glycoprotein',
'S glycoprotein',
'S peplomer polypeptide precursor',
'membrane associated glycoprotein E2 precursor',
'putative spike glycoprotein S',
'spike glycoprotein (S)',
'spike structural protein',
'surface glycoprotein S',
'surface projection glycoprotein' }

In [18]:
getgene("S_0529.fasta",S)

In [19]:

duplicate_remover("S_0529.fasta")

duplicate records removed!
11799 unique records saved from S_0529.fasta


In [4]:

## filter by length and propotion of "N"
## the minimal length of sequence

##from https://biopython.org/wiki/Sequence_Cleaner
def sequence_cleaner(fasta_file, min_length=0, por_n=100):
    output_file = open("clear_" + fasta_file, "w")

    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        if (len(seq_record.seq) >= min_length and (float((seq_record.seq).count("N"))/float(len(seq_record.seq)))*100 <= por_n):
            output_file.write(">" + seq_record.id + "\n" + str(seq_record.seq) + "\n")

    print("CLEAN!!!\nPlease check clear_" + fasta_file)

In [5]:
sequence_cleaner("rmS_0529.fasta",1000,50)

CLEAN!!!
Please check clear_rmS_0529.fasta


In [7]:
## get metadata table for S gene
meta = pd.read_csv("BetaCoV_0529.csv")
    
df = ph.read_fasta("rmS_0529.fasta")
df = df.filter(['id','sequence'], axis=1)
df[['Accession','version']] = df.id.str.split(".",expand=True)
fasta=pd.merge(df, meta, how='left', left_on='Accession', right_on='Accession')
fasta=fasta.drop(columns=['sequence'])
fasta.to_csv("BetaCoV_S_0529.csv",index = False)

In [ ]:
## get species fasta for with excel
